Objective: create a series of monthly 2D maps of ocean heat content anomaly produced by each mapping method

In [21]:
using DIVAnd
using Dates
using NCDatasets
using Test
using PyPlot
using Statistics
using DelimitedFiles
const plt = PyPlot
include("./ME4OH.jl")
include("./config.jl")

[ Info: Directory exists
[ Info: File already downloaded


In [22]:
ME4OH.get_filelist(datadirdisk)

160-element Vector{String}:
 "/media/ctroupin/T7 Shield/00006" ⋯ 83 bytes ⋯ "te.extra.danom.197901_201412.nc"
 "/media/ctroupin/T7 Shield/00006" ⋯ 83 bytes ⋯ "te.extra.danom.197901_201412.nc"
 "/media/ctroupin/T7 Shield/00006" ⋯ 83 bytes ⋯ "te.extra.danom.197901_201412.nc"
 "/media/ctroupin/T7 Shield/00006" ⋯ 83 bytes ⋯ "te.extra.danom.197901_201412.nc"
 "/media/ctroupin/T7 Shield/00006" ⋯ 83 bytes ⋯ "te.extra.danom.197901_201412.nc"
 "/media/ctroupin/T7 Shield/00006" ⋯ 83 bytes ⋯ "te.extra.danom.197901_201412.nc"
 "/media/ctroupin/T7 Shield/00006" ⋯ 83 bytes ⋯ "te.extra.danom.197901_201412.nc"
 "/media/ctroupin/T7 Shield/00006" ⋯ 83 bytes ⋯ "te.extra.danom.197901_201412.nc"
 "/media/ctroupin/T7 Shield/00006" ⋯ 83 bytes ⋯ "te.extra.danom.197901_201412.nc"
 "/media/ctroupin/T7 Shield/00006" ⋯ 83 bytes ⋯ "te.extra.danom.197901_201412.nc"
 "/media/ctroupin/T7 Shield/00006" ⋯ 83 bytes ⋯ "te.extra.danom.197901_201412.nc"
 "/media/ctroupin/T7 Shield/00006" ⋯ 83 bytes ⋯ "te.extra.danom.197901

## Set time period and read all the data

In [4]:
timeperiodtest = timeperiod1
datafilelist = ME4OH.get_filelist(datadir, timeperiodtest);
@info("Found $(length(datafilelist)) data files in $(datadir)")

[ Info: Found 2 data files in /home/ctroupin/data/ME4OH/data/en4.1.1/1979-2014/full/update/


In [5]:
@time obslon, obslat, obsdepth, obsdates, dohc = ME4OH.read_data_dohc(datafilelist);

  1.437726 seconds (3.63 M allocations: 253.627 MiB, 17.72% gc time, 96.58% compilation time)


[ Info: Found 18635 out of 26343 values
[ Info: Found 21679 out of 30789 values


## Creation of the background field
We have to define the time period for the climatology:
- full period for the years
- monthly periods.

In [8]:
yearlist = [timeperiod1[1]:timeperiod1[end]];
monthlist = [ii:ii for ii in 1:12]
TSbackground = DIVAnd.TimeSelectorYearListMonthList(yearlist,monthlist);
@show TSbackground;

TSbackground = TimeSelectorYearListMonthList{Vector{UnitRange{Int64}}, Vector{UnitRange{Int64}}}(UnitRange{Int64}[1979:2014], UnitRange{Int64}[1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:8, 9:9, 10:10, 11:11, 12:12])


### Set the analysis parameters

In [17]:
sz = (length(longrid), length(latgrid), 3);
lenx = zeros(Float32, sz)
leny = zeros(Float32, sz)
lenz = zeros(Float32, sz)
lenx[:,:,1] .= 12.5
lenx[:,:,2] .= 11.9
lenx[:,:,3] .= 19.9
leny[:,:,1] .= 12.5
leny[:,:,2] .= 11.9
leny[:,:,3] .= 19.9;
len = (lenx, leny, lenz)

(Float32[12.5 12.5 … 12.5 12.5; 12.5 12.5 … 12.5 12.5; … ; 12.5 12.5 … 12.5 12.5; 12.5 12.5 … 12.5 12.5;;; 11.9 11.9 … 11.9 11.9; 11.9 11.9 … 11.9 11.9; … ; 11.9 11.9 … 11.9 11.9; 11.9 11.9 … 11.9 11.9;;; 19.9 19.9 … 19.9 19.9; 19.9 19.9 … 19.9 19.9; … ; 19.9 19.9 … 19.9 19.9; 19.9 19.9 … 19.9 19.9], Float32[12.5 12.5 … 12.5 12.5; 12.5 12.5 … 12.5 12.5; … ; 12.5 12.5 … 12.5 12.5; 12.5 12.5 … 12.5 12.5;;; 11.9 11.9 … 11.9 11.9; 11.9 11.9 … 11.9 11.9; … ; 11.9 11.9 … 11.9 11.9; 11.9 11.9 … 11.9 11.9;;; 19.9 19.9 … 19.9 19.9; 19.9 19.9 … 19.9 19.9; … ; 19.9 19.9 … 19.9 19.9; 19.9 19.9 … 19.9 19.9], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [10]:
epsilon2 = 5.
solver = :direct

filenamebackground = "dohc_background_test.nc"
filename = "dohc_test.nc"

"dohc_test.nc"

In [11]:
depthgrid = [depthlayer1[2], depthlayer2[2], depthlayer3[2]]

3-element Vector{Float64}:
  286.6
  685.9
 1985.3

## Create grid
### Longitude / latitude
Defined in the file `config.jl`
### Depth levels
Read from the first file

In [13]:
_, _, _, depthgrid, _, _, _ = ME4OH.read_profile(datafilelist[1]);
depthgrid;

### Time periods

In [14]:
TS = TimeSelectorYearListMonthList([yy:yy for yy in timeperiodtest], [ii:ii for ii in 1:12]);
length(TS)

432

## Run analysis
### Set analysis parameters

In [19]:
sz = (length(longrid), length(latgrid), 3);
lenx = zeros(Float32, sz)
leny = zeros(Float32, sz)
lenz = zeros(Float32, sz)
lenx[:,:,1] .= 12.5
lenx[:,:,2] .= 11.9
lenx[:,:,3] .= 19.9
leny[:,:,1] .= 12.5
leny[:,:,2] .= 11.9
leny[:,:,3] .= 19.9;
len = (lenx, leny, lenz)
epsilon2 = 5.

5.0

In [99]:
obslonall, obslatall, obsdepthall = Float64.(obslonall), Float64.(obslatall), Float64.(obsdepthall)
Tall = Float64.(Tall);

In [16]:
@time dbinfo = diva3d((longrid, latgrid, depthgrid, TS),
              (obslonall, obslatall, obsdepthall, obsdatesall),
              Tall,
              (lenx,leny,lenz),
              epsilon2,
              joinpath(outputdir, "diva3d2delete.nc"), "Temperature",
              bathname = gebco16file,
              bathisglobal = true,
              background = DIVAnd.backgroundfile(filenamebackground, "dohc", TSbackground),
       );

LoadError: UndefVarError: `obslonall` not defined